In [3]:
from EZ.word import word_DVs, word_means
from EZ.sentence import Sentence
from EZ.fixation import Fixation

import os
import pandas as pd

In [4]:
word_info_df = pd.read_csv('/data/home/shared/onestop/OneStop_v1_20250126/lacclab_processed/ia_Paragraph.csv', engine="pyarrow")
word_info_df = word_info_df[word_info_df["article_id"] != 0] # filter out practice article
word_info_df = word_info_df[word_info_df["repeated_reading_trial"] == 0] # use only non repeated reading
word_info_df = word_info_df[word_info_df["question_preview"] == 0] # use only non question preview cases
word_info_df['IA_LABEL'] = word_info_df.IA_LABEL.replace('\t(.*)', '', regex=True)

In [5]:
paragraphs = word_info_df.groupby(['unique_paragraph_id', 'text_spacing_version'])['word_length'].apply(list).reset_index()
paragraphs

,unique_paragraph_id,text_spacing_version,word_length
0,1_10_Adv_1,0,"[5, 10, 2, 2, 7, 2, 9, 2, 3, 6, 3, 10, 3, 4, 2..."
1,1_10_Adv_1,1,"[5, 10, 2, 2, 7, 2, 9, 2, 3, 6, 3, 10, 3, 4, 2..."
2,1_10_Adv_2,0,"[4, 7, 3, 6, 5, 4, 2, 5, 3, 7, 2, 5, 10, 3, 4,..."
3,1_10_Adv_3,0,"[6, 3, 8, 3, 10, 3, 1, 5, 5, 4, 4, 9, 2, 3, 6,..."
4,1_10_Adv_4,0,"[7, 2, 4, 6, 11, 7, 4, 4, 4, 9, 4, 11, 1, 5, 5..."
...,...,...,...
334,3_9_Ele_1,0,"[3, 5, 4, 5, 4, 1, 10, 3, 7, 2, 3, 5, 4, 7, 2,..."
335,3_9_Ele_2,0,"[3, 2, 6, 6, 1, 16, 12, 6, 3, 6, 2, 5, 2, 5, 2..."
336,3_9_Ele_3,0,"[5, 4, 3, 7, 3, 7, 3, 9, 2, 4, 5, 6, 3, 3, 2, ..."
337,3_9_Ele_3,1,"[5, 4, 3, 7, 3, 7, 3, 9, 2, 4, 5, 6, 3, 3, 2, ..."


In [6]:
# Define the folder path
folder_path = "results/Eyettention"

# Get a list of all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Dictionary to store DataFrames
eyettention_outputs = {}

# Loop through each CSV file and load it into a DataFrame
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    eyettention_outputs[file] = df  # Store DataFrame in dictionary with filename as key

In [7]:
paragraphs_dict = {}
for i, row in paragraphs.iterrows():
    paragraphs_dict[(row["unique_paragraph_id"], row["text_spacing_version"])] = Sentence(i, row["word_length"])

In [8]:
def split_df_on_zero(df, column_name='fix_id'):
    # Find indices where the column equals 0
    df = df.reset_index()
    split_indices = df.index[df[column_name] == 0].tolist()
    # Add end index
    split_indices.append(len(df))
    # Create a list of DataFrames
    dfs = [df.iloc[split_indices[i]:split_indices[i+1]].reset_index(drop=True) for i in range(len(split_indices)-1)]
    return dfs

In [9]:
fold_dict = {}
for fold_output in sorted(eyettention_outputs.keys()):
    processed_fold = {}
    output_df = eyettention_outputs[fold_output]
    grouped_dfs = {key: group for key, group in output_df.groupby(['unique_paragraph_id', 'text_spacing_version'])}
    grouped_dfs = {key: split_df_on_zero(group) for key, group in grouped_dfs.items()}
    for key, group in grouped_dfs.items():
        processed_group = []
        for df in group:
            processed_df = []
            for i, row in df.iterrows():
                processed_df.append(Fixation(row["sp_fix_dur"], row["fix_id"], row["sp_fix_pos"]))
            processed_group.append(processed_group)
        processed_fold[key] = processed_group
    fold_dict[fold_output] = processed_fold


In [ ]:
for file_name, combo_dict in fold_dict.items():
    for combo, fix_lists in combo_dict.items():
        paragraph = paragraphs_dict[combo]
        for fix_list in fix_lists:
            adjusted_paragraph = word_DVs(paragraph, fix_list)
            print('Hi')
            break
        break
    break

AttributeError: 'list' object has no attribute 'word'

AttributeError: 'list' object has no attribute 'word'

AttributeError: 'list' object has no attribute 'word'